In [1]:
import pandas as pd
test_data = pd.read_csv("./Data/corrected_novel_words.csv")
test_data.head()

c:\Users\river\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\river\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\river\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


,Column1,sentence,new_word,Tokenizer,Label,correct,my_correction,my_correction_pos,my_correction_pos2,Jaccard_correction,edit_distance,mix_jaccard
0,0,موضوع الإشاعة بتاعة ده لازم يتجاب سليم وحتة وا...,وحتة,"['وحت', '##ة']",1,"{'وحتة': ['حتة', 'وحة', 'وحت', 'وتة', 'وحتل', ...",True,True,True,True,True,True
1,1,كنت قاعد بتكلم انا وبابا فمره فحوار اني علي طو...,فمره,"['فمر', '##ه']",1,"{'فمره': ['مره', 'عمره', 'خمره', 'فكره']}",True,True,True,True,True,True
2,2,كنت قاعد بتكلم انا وبابا فمره فحوار اني علي طو...,فحوار,"['فح', '##وار']",1,"{'فحوار': ['وار', 'حوا', 'فتحوا', 'فار', 'حار'...",True,True,True,True,True,True
3,3,اول مرة اطلب قهوة في قهوة فقالي سادة قولتلة ما...,فقالي,"['فقال', '##ي']",1,"{'فقالي': ['قالي', 'فهالي', 'فعالي', 'تقالي', ...",True,True,True,True,True,True
4,4,لحد البيت بتاعنا اتهد,اتهد,"['اته', '##د']",1,"{'اتهد': ['اتد', 'اهد', 'تهد', 'اتبهد', 'اته',...",True,True,True,True,True,True


In [2]:
import marisa_trie
MARBert_trie = marisa_trie.Trie().load("../Data/Dictionary/MARBert_trie_dictionary.marisa")

In [4]:
from Misspelled_correction import ArabicCorrector,correct
corrector = ArabicCorrector(MARBert_trie.keys())

## Method 1

In [ ]:
corrected = []
for word in test_data.new_word.values:
    if word in MARBert_trie:
        corrected.append(True)
    else:
        corrected.append(corrector.get_most_similar(word))

In [7]:
test_data["my_correction"] = corrected

In [62]:
test_data.to_csv("./Data/corrected_novel_words.csv",index=False)

## Method 2

In [37]:
import re

corrected_2 = []
for word in test_data.new_word.values:
    half = word[:len(word)//2]
    pre_suf = MARBert_trie.prefixes(half)+ MARBert_trie.keys(half)

    if word in MARBert_trie:
        corrected_2.append(True)
    elif word in pre_suf:
        corrected_2.append(pre_suf.index(word))
    else:
        word = re.sub(r"ة$","ه",word)
        word = re.sub(r"آ$","ء",word)
        word = re.sub(r"أ$","ء",word)
        word = re.sub(r"إ$","ء",word)
        corrector_neighbors = ArabicCorrector(pre_suf)
        corrected_neighbor = corrector_neighbors.get_most_similar(word,metric=ArabicCorrector.cosin_similarity_postional_embedding_ave)
        if corrected_neighbor[0][0]>=.999:
            corrected_2.append(corrected_neighbor)
            print(word,corrected_neighbor)
            continue
        corrected_all = corrector.get_most_similar(word,metric=ArabicCorrector.cosin_similarity_postional_embedding_ave)

        corrected_2.append(corrected_all if corrected_all[0][0]> corrected_neighbor[0][0] else corrected_neighbor)

مروتى [(0.9999700712498871, 'مرتوي')]
هتعوف [(0.999881508156015, 'هتوفع')]
الويسخ [(0.999948468562027, 'الوسيخ')]
يتكرش [(0.999881508156015, 'يتشكر')]
مفحوت [(0.9998428580253238, 'مفتوح')]


In [50]:
import re
# w = re.sub(r"ة$","ه","فاحصة")
# w = re.sub(r"آ$","ء","فاحصة")
# w = re.sub(r"أ$","ء","فاحصة")
# w = re.sub(r"إ$","ء","فاحصة")
import itertools
import numpy as np

class Syntactic_Symilarity:
    def __init__(self, data):
        self.corpus = data
    def jaccard_set(self, a, b):
        # convert to set
        a = set(a)
        b = set(b)
        # calucate jaccard similarity
        j = float(len(a.intersection(b))) / len(a.union(b))
        return j

    def syntactic_similarity(self, symp_t):
        most_sim=[]
        poss_sym=[]
        for symp in self.corpus:
            d=self.jaccard_set(symp_t,symp)
            most_sim.append(d)
        
        order=np.argsort(most_sim)[::-1].tolist()
        return most_sim[order[0]], self.corpus[order[0]]

semantic = Syntactic_Symilarity(MARBert_trie.keys())
w = "مافرقش"
r = "مشفارق"
r2 = "فاحترمني"
print(ArabicCorrector.cosin_similarity_postional_embedding_ave(w,r))
print(semantic.syntactic_similarity("مافرحتيني"))
# corrector.get_most_similar(w,metric=ArabicCorrector.cosin_similarity_postional_embedding_ave)

0.9967224621222924
(1.0, 'محترفينا')


In [54]:
from nltk import edit_distance
edit_distance(r,w)

3

In [ ]:
test_data["my_correction_pos2"] = corrected_2
test_data[~(test_data.my_correction_pos==True)]
test_data

In [59]:
test_data.drop(columns="Unnamed: 0" ,inplace=True)

In [60]:
test_data[~(test_data.my_correction_pos==True)]

,Column1,sentence,new_word,Tokenizer,Label,correct,my_correction,my_correction_pos,my_correction_pos2
6,6,ال انا مافرقش معاهم هما برده مايفرقوش معايا,مافرقش,"['مافر', '##قش']",1,"{'مافرقش': ['مفرش', 'مافبش', 'مافيش', 'مفروش',...","[(1.0000000000000002, 'مشفارق')]","[(0.9997949422029054, مشفارق)]","[(0.9997949422029054, مشفارق)]"
7,7,مفيش زكرا هترجعني,زكرا,"['زك', '##را']",1,"{'زكرا': ['زرا', 'كرا', 'سكرا', 'وكرا', 'زورا'...","[(1.0, 'آركز')]","[(0.9999937363683398, آركز)]","[(0.9999937363683398, آركز)]"
9,9,ياخراشي ع مروتى وجمالها وطيبة قلبها محبتش تزعل...,مروتى,"['مرو', '##تى']",1,"{'مروتى': ['متى', 'مرو', 'مرت', 'موت', 'روت', ...","[(0.9999999999999998, 'ريموت')]","[(0.9999999349812038, مورتى)]","[(0.9999700712498871, مرتوي)]"
10,10,ياخراشي ع مروتى وجمالها وطيبة قلبها محبتش تزعل...,زالفل,"['زال', '##فل']",1,"{'زالفل': ['الفل', 'بالفل']}","[(0.9999999999999999, 'الزلف')]","[(0.9999083266897983, الزلف)]","[(0.9999083266897983, الزلف)]"
11,11,ياخراشي ع مروتى وجمالها وطيبة قلبها محبتش تزعل...,مافرحتيني,"['مافر', '##حتين', '##ي']",1,{'مافرحتيني': ['مارتيني']},"[(0.9594032236002469, 'فاحترمني')]","[(0.9983816131593496, فاحترمني)]","[(0.9983816131593496, فاحترمني)]"
13,13,لو مش هتندم مش هتعلم ومش هتعوف انك غلط وان فين...,هتعوف,"['هتع', '##وف']",1,{'هتعوف': ['هتشوف']},"[(0.9999999999999998, 'تفعوه')]","[(0.9999793493765022, فعتوه)]","[(0.999881508156015, هتوفع)]"
15,15,اسكت بقا قولت النهار هيبان مبانشي قولت الامل ه...,مبانشي,"['مبا', '##نشي']",1,{'مبانشي': ['مباني']},"[(1.0000000000000002, 'شامبين')]","[(0.9999999328729082, مابنشى)]","[(0.9999999328729082, مابنشى)]"
18,18,الزمالكاوبه ملهم عمالين يقولولك غور ارحل اتنيل...,الزمالكاوبه,"['الزمالك', '##اوب', '##ه']",1,{'الزمالكاوبه': ['لم يتم العثور علي إقتراحات م...,"[(0.9746794344808963, 'باكمالهالفوز')]","[(0.9965691266925233, الزملكوبه)]","[(0.9965691266925233, الزملكوبه)]"
19,19,الزمالكاوبه ملهم عمالين يقولولك غور ارحل اتنيل...,مايمون,"['مايم', '##ون']",1,"{'مايمون': ['مايون', 'مايكون']}","[(0.9999999999999998, 'أمنيوم')]","[(0.9999744547046225, ومأيمن)]","[(0.9999744547046225, ومأيمن)]"
20,20,عايزة حد يعملي غسيل مخ عشان ينضف من المنهج الو...,الويسخ,"['الوي', '##سخ']",1,"{'الويسخ': ['الويس', 'الويسك', 'الويست']}","[(1.0000000000000002, 'الوسيخ')]","[(0.999948468562027, الوسيخ)]","[(0.999948468562027, الوسيخ)]"


# Jaccard

In [36]:
import itertools
import numpy as np

class Syntactic_Symilarity:
    def __init__(self, data):
        self.corpus = data
    def jaccard_set(self, a, b):
        # convert to set
        a = set(a)
        b = set(b)
        # calucate jaccard similarity
        j = float(len(a.intersection(b))) / len(a.union(b))
        return j

    def syntactic_similarity(self, symp_t):
        most_sim=[]
        for symp in self.corpus:
            d=self.jaccard_set(symp_t,symp)
            most_sim.append(d)
        
        order=np.argsort(most_sim)[::-1].tolist()
        matched_word=[(most_sim[order[0]], self.corpus[order[0]])]
        for i in range(1,len(order)):
            if most_sim[order[i-1]] != most_sim[order[i]]:
                break
            matched_word.append((most_sim[order[i]], self.corpus[order[i]]))
        return matched_word

syntactic = Syntactic_Symilarity(MARBert_trie.keys())

In [ ]:

corrected_3 = []
for word in test_data.new_word.values:
    half = word[:len(word)//2]
    pre_suf = MARBert_trie.prefixes(half)+ MARBert_trie.keys(half)

    if word in MARBert_trie:
        corrected_3.append(True)
    elif word in pre_suf:
        corrected_3.append(pre_suf.index(word))
    else:
        # word = re.sub(r"ة$","ه",word)
        # word = re.sub(r"آ$","ء",word)
        # word = re.sub(r"أ$","ء",word)
        # word = re.sub(r"إ$","ء",word)
        # syntactic2  = Syntactic_Symilarity(pre_suf)
        # corrected_neighbor = syntactic2.syntactic_similarity(word)
        # if corrected_neighbor[0][0]>=.9999:
        #     corrected_3.append(corrected_neighbor)
        #     print(word,corrected_neighbor)
        #     continue
        corrected_all = syntactic.syntactic_similarity(word)

        corrected_3.append(corrected_all )#if corrected_all[0][0]> corrected_neighbor[0][0] else corrected_neighbor)

In [14]:
test_data["Jaccard_correction"] = corrected_3
test_data[~(test_data.Jaccard_correction==True)]

,Column1,sentence,new_word,Tokenizer,Label,correct,my_correction,my_correction_pos,my_correction_pos2,Jaccard_correction
6,6,ال انا مافرقش معاهم هما برده مايفرقوش معايا,مافرقش,"['مافر', '##قش']",1,"{'مافرقش': ['مفرش', 'مافبش', 'مافيش', 'مفروش',...","[(1.0000000000000002, 'مشفارق')]","[(0.9997949422029054, 'مشفارق')]","[(0.9997949422029054, 'مشفارق')]","[[1.0, مشفارق]]"
7,7,مفيش زكرا هترجعني,زكرا,"['زك', '##را']",1,"{'زكرا': ['زرا', 'كرا', 'سكرا', 'وكرا', 'زورا'...","[(1.0, 'آركز')]","[(0.9999937363683398, 'آركز')]","[(0.9999937363683398, 'آركز')]","[[1.0, زكراك]]"
9,9,ياخراشي ع مروتى وجمالها وطيبة قلبها محبتش تزعل...,مروتى,"['مرو', '##تى']",1,"{'مروتى': ['متى', 'مرو', 'مرت', 'موت', 'روت', ...","[(0.9999999999999998, 'ريموت')]","[(0.9999999349812038, 'مورتى')]","[(0.9999700712498871, 'مرتوي')]","[[1.0, وترتمى]]"
10,10,ياخراشي ع مروتى وجمالها وطيبة قلبها محبتش تزعل...,زالفل,"['زال', '##فل']",1,"{'زالفل': ['الفل', 'بالفل']}","[(0.9999999999999999, 'الزلف')]","[(0.9999083266897983, 'الزلف')]","[(0.9999083266897983, 'الزلف')]","[[1.0, زالف]]"
11,11,ياخراشي ع مروتى وجمالها وطيبة قلبها محبتش تزعل...,مافرحتيني,"['مافر', '##حتين', '##ي']",1,{'مافرحتيني': ['مارتيني']},"[(0.9594032236002469, 'فاحترمني')]","[(0.9983816131593496, 'فاحترمني')]","[(0.9983816131593496, 'فاحترمني')]","[[1.0, مافرحتينا]]"
13,13,لو مش هتندم مش هتعلم ومش هتعوف انك غلط وان فين...,هتعوف,"['هتع', '##وف']",1,{'هتعوف': ['هتشوف']},"[(0.9999999999999998, 'تفعوه')]","[(0.9999793493765022, 'فعتوه')]","[(0.999881508156015, 'هتوفع')]","[[1.0, هتوفع]]"
15,15,اسكت بقا قولت النهار هيبان مبانشي قولت الامل ه...,مبانشي,"['مبا', '##نشي']",1,{'مبانشي': ['مباني']},"[(1.0000000000000002, 'شامبين')]","[(0.9999999328729082, 'مابنشى')]","[(0.9999999328729082, 'مابنشى')]","[[1.0, مابينمش]]"
18,18,الزمالكاوبه ملهم عمالين يقولولك غور ارحل اتنيل...,الزمالكاوبه,"['الزمالك', '##اوب', '##ه']",1,{'الزمالكاوبه': ['لم يتم العثور علي إقتراحات م...,"[(0.9746794344808963, 'باكمالهالفوز')]","[(0.9965691266925233, 'الزملكوبه')]","[(0.9965691266925233, 'الزملكوبه')]","[[1.0, الزملكوبه]]"
19,19,الزمالكاوبه ملهم عمالين يقولولك غور ارحل اتنيل...,مايمون,"['مايم', '##ون']",1,"{'مايمون': ['مايون', 'مايكون']}","[(0.9999999999999998, 'أمنيوم')]","[(0.9999744547046225, 'ومأيمن')]","[(0.9999744547046225, 'ومأيمن')]","[[1.0, ماينوي]]"
20,20,عايزة حد يعملي غسيل مخ عشان ينضف من المنهج الو...,الويسخ,"['الوي', '##سخ']",1,"{'الويسخ': ['الويس', 'الويسك', 'الويست']}","[(1.0000000000000002, 'الوسيخ')]","[(0.999948468562027, 'الوسيخ')]","[(0.999948468562027, 'الوسيخ')]","[[1.0, الوسيخ]]"


## Edit distance

In [27]:
from nltk import edit_distance

def my_edit_distance(word,corpus = MARBert_trie.keys()):
    matched_word  = []
    min = np.Infinity
    for w in corpus:
        d = edit_distance(word,w)
        if d <min:
            min = d
            matched_word=[(d,w)]
        elif d== min:
            matched_word.append((d,w))


    return matched_word


In [26]:
corrected_4 = []
for word in test_data.new_word.values:
    half = word[:len(word)//2]
    pre_suf = MARBert_trie.prefixes(half)+ MARBert_trie.keys(half)

    if word in MARBert_trie:
        corrected_4.append(True)
    elif word in pre_suf:
        corrected_4.append(pre_suf.index(word))
    else:
        # word = re.sub(r"ة$","ه",word)
        # word = re.sub(r"آ$","ء",word)
        # word = re.sub(r"أ$","ء",word)
        # word = re.sub(r"إ$","ء",word)
        # if word in MARBert_trie:
        #     corrected_4.append(word)
        #     continue
        # syntactic2  = Syntactic_Symilarity(pre_suf)
        corrected_neighbor = my_edit_distance(word,pre_suf)
        if corrected_neighbor[0][0] == 1:
            corrected_4.append(corrected_neighbor)
            print(word,corrected_neighbor)
            continue
        corrected_all = my_edit_distance(word)

        corrected_4.append(corrected_all if corrected_all[0][0]< corrected_neighbor[0][0] else corrected_neighbor)

مافرقش [(1, 'مافاقش')]
زكرا [(1, 'زكر')]
مروتى [(1, 'مراتى')]
زالفل [(1, 'زالزل')]
مافرحتيني [(1, 'مافرحتينا')]
هتعوف [(1, 'هتعرف')]
مبانشي [(1, 'مباطشي')]
الزمالكاوبه [(1, 'الزمالكاويه')]
مايمون [(1, 'مايتون')]
الويسخ [(1, 'الوسخ')]
نضارتي [(1, 'نضارته')]
يتكرش [(1, 'يتحرش')]
مفحوت [(1, 'مفحور')]
اضاضا [(1, 'اضاؤا')]
ماحصة [(1, 'ماحصل')]
تملقك [(1, 'تملك')]


In [29]:
test_data["edit_distance"] = corrected_4
test_data[~(test_data.Jaccard_correction==True)][["new_word","my_correction","my_correction_pos","my_correction_pos2","Jaccard_correction","edit_distance"]]

,new_word,my_correction,my_correction_pos,my_correction_pos2,Jaccard_correction,edit_distance
6,مافرقش,"[(1.0000000000000002, 'مشفارق')]","[(0.9997949422029054, 'مشفارق')]","[(0.9997949422029054, 'مشفارق')]","[[1.0, مشفارق]]","[(1, مافاقش)]"
7,زكرا,"[(1.0, 'آركز')]","[(0.9999937363683398, 'آركز')]","[(0.9999937363683398, 'آركز')]","[[1.0, زكراك]]","[(1, زكر)]"
9,مروتى,"[(0.9999999999999998, 'ريموت')]","[(0.9999999349812038, 'مورتى')]","[(0.9999700712498871, 'مرتوي')]","[[1.0, وترتمى]]","[(1, مراتى)]"
10,زالفل,"[(0.9999999999999999, 'الزلف')]","[(0.9999083266897983, 'الزلف')]","[(0.9999083266897983, 'الزلف')]","[[1.0, زالف]]","[(1, زالزل)]"
11,مافرحتيني,"[(0.9594032236002469, 'فاحترمني')]","[(0.9983816131593496, 'فاحترمني')]","[(0.9983816131593496, 'فاحترمني')]","[[1.0, مافرحتينا]]","[(1, مافرحتينا)]"
13,هتعوف,"[(0.9999999999999998, 'تفعوه')]","[(0.9999793493765022, 'فعتوه')]","[(0.999881508156015, 'هتوفع')]","[[1.0, هتوفع]]","[(1, هتعرف)]"
15,مبانشي,"[(1.0000000000000002, 'شامبين')]","[(0.9999999328729082, 'مابنشى')]","[(0.9999999328729082, 'مابنشى')]","[[1.0, مابينمش]]","[(1, مباطشي)]"
18,الزمالكاوبه,"[(0.9746794344808963, 'باكمالهالفوز')]","[(0.9965691266925233, 'الزملكوبه')]","[(0.9965691266925233, 'الزملكوبه')]","[[1.0, الزملكوبه]]","[(1, الزمالكاويه)]"
19,مايمون,"[(0.9999999999999998, 'أمنيوم')]","[(0.9999744547046225, 'ومأيمن')]","[(0.9999744547046225, 'ومأيمن')]","[[1.0, ماينوي]]","[(1, مايتون)]"
20,الويسخ,"[(1.0000000000000002, 'الوسيخ')]","[(0.999948468562027, 'الوسيخ')]","[(0.999948468562027, 'الوسيخ')]","[[1.0, الوسيخ]]","[(1, الوسخ)]"


# Mix

In [5]:
corrected_5 = []
for word in test_data.new_word.values:
    half = word[:len(word)//2]
    pre_suf = MARBert_trie.prefixes(half)+ MARBert_trie.keys(half)

    if word in MARBert_trie:
        corrected_5.append(True)
    elif word in pre_suf:
        corrected_5.append(pre_suf.index(word))
    else:
   
        syntactic2  = Syntactic_Symilarity(pre_suf)
        corrected_neighbor = syntactic2.syntactic_similarity(word)
        if corrected_neighbor[0][0] == 1:
            words = [wo[1] for wo in corrected_neighbor]
            temp  = ArabicCorrector(words)
            corr = temp.get_most_similar(word,metric=ArabicCorrector.cosin_similarity_postional_embedding_ave)
            corrected_5.append(corr)
            print(word, corr)
            continue
        corrected_all = syntactic.syntactic_similarity(word)
        words = [wo[1] for wo in corrected_all]
        temp  = ArabicCorrector(words)
        corr = temp.get_most_similar(word,metric=ArabicCorrector.cosin_similarity_postional_embedding_ave)
        corrected_5.append(corr)
        # corrected_5.append(corrected_all if corrected_all[0][0]< corrected_neighbor[0][0] else corrected_neighbor)

[(0.9984812901763878, 'زكراك')]

In [61]:
test_data["mix_jaccard"] = corrected_5
test_data[~(test_data.Jaccard_correction==True)][["new_word","my_correction","my_correction_pos","my_correction_pos2","Jaccard_correction","edit_distance","mix_jaccard"]]

,new_word,my_correction,my_correction_pos,my_correction_pos2,Jaccard_correction,edit_distance,mix_jaccard
6,مافرقش,"[(1.0000000000000002, 'مشفارق')]","[(0.9997949422029054, 'مشفارق')]","[(0.9997949422029054, 'مشفارق')]","[[1.0, مشفارق]]","[(1, مافاقش)]","[(0.9997949422029054, مشفارق)]"
7,زكرا,"[(1.0, 'آركز')]","[(0.9999937363683398, 'آركز')]","[(0.9999937363683398, 'آركز')]","[[1.0, زكراك]]","[(1, زكر)]","[(0.9984812901763878, زكراك)]"
9,مروتى,"[(0.9999999999999998, 'ريموت')]","[(0.9999999349812038, 'مورتى')]","[(0.9999700712498871, 'مرتوي')]","[[1.0, وترتمى]]","[(1, مراتى)]","[(0.9999999349812038, مورتى)]"
10,زالفل,"[(0.9999999999999999, 'الزلف')]","[(0.9999083266897983, 'الزلف')]","[(0.9999083266897983, 'الزلف')]","[[1.0, زالف]]","[(1, زالزل)]","[(0.9984740635109888, زالف)]"
11,مافرحتيني,"[(0.9594032236002469, 'فاحترمني')]","[(0.9983816131593496, 'فاحترمني')]","[(0.9983816131593496, 'فاحترمني')]","[[1.0, مافرحتينا]]","[(1, مافرحتينا)]","[(0.9967224621222927, مافرحتينا)]"
13,هتعوف,"[(0.9999999999999998, 'تفعوه')]","[(0.9999793493765022, 'فعتوه')]","[(0.999881508156015, 'هتوفع')]","[[1.0, هتوفع]]","[(1, هتعرف)]","[(0.999881508156015, هتوفع)]"
15,مبانشي,"[(1.0000000000000002, 'شامبين')]","[(0.9999999328729082, 'مابنشى')]","[(0.9999999328729082, 'مابنشى')]","[[1.0, مابينمش]]","[(1, مباطشي)]","[(0.9998875040665968, مابينش)]"
18,الزمالكاوبه,"[(0.9746794344808963, 'باكمالهالفوز')]","[(0.9965691266925233, 'الزملكوبه')]","[(0.9965691266925233, 'الزملكوبه')]","[[1.0, الزملكوبه]]","[(1, الزمالكاويه)]","[(0.9965691266925233, الزملكوبه)]"
19,مايمون,"[(0.9999999999999998, 'أمنيوم')]","[(0.9999744547046225, 'ومأيمن')]","[(0.9999744547046225, 'ومأيمن')]","[[1.0, ماينوي]]","[(1, مايتون)]","[(0.9987966086142227, مايون)]"
20,الويسخ,"[(1.0000000000000002, 'الوسيخ')]","[(0.999948468562027, 'الوسيخ')]","[(0.999948468562027, 'الوسيخ')]","[[1.0, الوسيخ]]","[(1, الوسخ)]","[(0.999948468562027, الوسيخ)]"
